In [1]:
import os
import cmdstanpy
from cmdstanpy import Model, StanFit, cmdstan_path
from cmdstanpy import Model, StanFit
import ujson as json

<h3>Example:</h3>

Running the model on Men's 2014 World Cup

In [2]:
with open('../data/worldcup_gelman/worldcup_2014.data.json', 'r') as fd:
    data_dict = json.load(fd)
data_dict.keys()

# stan_wc = Model(stan_file='../data/worldcup_gelman/worldcup_rep_ag.stan')
# stan_wc.compile()
# stan_wc

# worldcup_fit = stan_wc.sample(data=data_dict, chains=4)
# worldcup_fit.diagnose()
# worldcup_fit.summary()

dict_keys(['I', 'N', 'team_1', 'team_2', 'score_1', 'score_2', 'prior_score'])

In [5]:
with open('../data/worldcup_gelman/countries.data.json', 'r') as fd_country:
    country_dict = json.load(fd_country)
country_dict.keys()

dict_keys(['countries'])

In [6]:
country_dict['countries'][0]

[1, 'Bresil']

<h3> Attempting the same for the Women' World Cup Data </h3>

(Import the data... and massaging it into the format stan needs it in.) 

Data input needs to be in a dictionary format;
Keys in the dictionary are the names of the variables in the data block of the stan model;

In [7]:
import pandas as pd
matches = pd.read_csv("/Users/nitya/NYCPyLadies_CmdStanPy/data/womens-world-cup-2019/wwc_matches.csv")
forecasts = pd.read_csv("/Users/nitya/NYCPyLadies_CmdStanPy/data/womens-world-cup-2019/wwc_forecasts.csv")
print(matches.columns, forecasts.columns)

Index(['date', 'league_id', 'league', 'team1', 'team2', 'spi1', 'spi2',
       'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2', 'score1',
       'score2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1', 'adj_score2'],
      dtype='object') Index(['forecast_timestamp', 'team', 'group', 'spi', 'global_o', 'global_d',
       'sim_wins', 'sim_ties', 'sim_losses', 'sim_goal_diff', 'goals_scored',
       'goals_against', 'group_1', 'group_2', 'group_3', 'group_4',
       'make_round_of_16', 'make_quarters', 'make_semis', 'make_final',
       'win_league', 'timestamp'],
      dtype='object')


In [8]:
matches.team2.unique()

array(['South Korea Women', 'China Women', 'South Africa Women',
       'Nigeria Women', 'Italy Women', 'Jamaica Women', 'Scotland Women',
       'Japan Women', 'Cameroon Women', 'Netherlands Women',
       'Sweden Women', 'Thailand Women', 'Spain Women', 'Norway Women',
       'Brazil Women', 'Argentina Women', 'New Zealand Women',
       'Chile Women', 'Germany Women', 'France Women', 'Australia Women',
       'England Women', 'Canada Women', 'USA Women'], dtype=object)

In [28]:
countries = forecasts[95:][['team', 'spi']].sort_values(by=['spi'], ascending=False)

In [29]:
wwc2019_country = countries['team'].values
print(wwc2019_country)

['USA' 'France' 'Germany' 'Australia' 'Netherlands' 'England' 'Japan'
 'Canada' 'Brazil' 'Sweden' 'Spain' 'New Zealand' 'Norway' 'China'
 'South Korea' 'Italy' 'Nigeria' 'Cameroon' 'South Africa' 'Jamaica'
 'Scotland' 'Thailand' 'Chile' 'Argentina' 'Argentina']


In [12]:
spi = dict(zip(wwc2019_country, list(range(1, len(wwc2019_country)))))

In [13]:
spi['Argentina'] = 24
spi

{'USA': 1,
 'France': 2,
 'Germany': 3,
 'Australia': 4,
 'Netherlands': 5,
 'England': 6,
 'Japan': 7,
 'Canada': 8,
 'Brazil': 9,
 'Sweden': 10,
 'Spain': 11,
 'New Zealand': 12,
 'Norway': 13,
 'China': 14,
 'South Korea': 15,
 'Italy': 16,
 'Nigeria': 17,
 'Cameroon': 18,
 'South Africa': 19,
 'Jamaica': 20,
 'Scotland': 21,
 'Thailand': 22,
 'Chile': 23,
 'Argentina': 24}

In [14]:
import numpy as np

In [15]:
len(countries['spi'])

24

In [16]:
mean = np.mean(countries['spi'])
std = np.std(countries['spi'])
countries['prior_score'] = [(x - mean)/std for x in countries['spi']]

In [17]:
std/mean

0.22513875698459

In [18]:
countries['prior_score']

97     1.159123
96     1.050296
98     0.984274
101    0.924374
100    0.899969
99     0.899669
102    0.875904
103    0.872143
105    0.693165
104    0.506575
106    0.374033
109    0.339315
107    0.335476
108    0.288647
110    0.211270
112   -0.344675
111   -0.356842
114   -0.839073
113   -0.897740
115   -0.997218
116   -1.517592
117   -1.535829
119   -1.620591
118   -2.304670
Name: prior_score, dtype: float64

In [19]:
matches['team1'] = matches['team1'].apply(lambda x: x.split(' ')[0])
matches['team2'] = matches['team2'].apply(lambda x: x.split(' ')[0])

In [20]:
mydict = dict({'team_1': matches['team1'].values,
               'team_2': matches['team2'].values,
               'score_1': matches['score1'].values,
               'score_2': matches['score2'].values,
              'prior_score': countries['prior_score'].values})

In [21]:
mydict['team_1'] = [spi.get(x) for x in mydict['team1']]
mydict['team_2'] = [spi.get(x) for x in mydict['team2']]

In [23]:
stan_wc = Model(stan_file='../data/worldcup_gelman/worldcup_rep_ag.stan')
stan_wc.compile()
stan_wc

INFO:cmdstanpy:compiling c++
INFO:cmdstanpy:compiled model file: /Users/nitya/NYCPyLadies_CmdStanPy/data/worldcup_gelman/worldcup_rep_ag


Model(name=worldcup_rep_ag,  stan_file="../data/worldcup_gelman/worldcup_rep_ag.stan", exe_file="/Users/nitya/NYCPyLadies_CmdStanPy/data/worldcup_gelman/worldcup_rep_ag")

In [24]:
mydict

{'team1': [2,
  3,
  11,
  13,
  4,
  9,
  6,
  24,
  8,
  None,
  23,
  1,
  17,
  3,
  2,
  4,
  None,
  7,
  20,
  6,
  5,
  8,
  10,
  1,
  14,
  None,
  17,
  None,
  16,
  20,
  21,
  7,
  18,
  5,
  10,
  22,
  3,
  13,
  6,
  2,
  11,
  10,
  16,
  5,
  13,
  2,
  16,
  3],
 'team2': [None,
  14,
  None,
  17,
  16,
  20,
  21,
  7,
  18,
  5,
  10,
  22,
  None,
  11,
  13,
  9,
  14,
  21,
  16,
  24,
  18,
  None,
  22,
  23,
  11,
  3,
  2,
  13,
  9,
  4,
  24,
  6,
  None,
  8,
  1,
  23,
  17,
  4,
  18,
  9,
  1,
  8,
  14,
  7,
  6,
  1,
  5,
  10],
 'score_1': array([ 4.,  1.,  3.,  3.,  1.,  3.,  2.,  0.,  1.,  0.,  0., 13.,  2.,
         1.,  2.,  3.,  0.,  2.,  0.,  1.,  3.,  2.,  5.,  3.,  0.,  0.,
         0.,  1.,  0.,  1.,  3.,  0.,  2.,  2.,  0.,  0.,  3.,  1.,  3.,
         2.,  1.,  1.,  2.,  2., nan, nan, nan, nan]),
 'score_2': array([ 0.,  0.,  1.,  0.,  2.,  0.,  1.,  0.,  0.,  1.,  2.,  0.,  0.,
         0.,  1.,  2.,  1.,  1.,  5.,  0.,  1.,  0.,  1., 

In [27]:
worldcup_fit = stan_wc.sample(data=mydict, chains=4)
worldcup_fit.diagnose()
worldcup_fit.summary()

INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:finish chain 4


RuntimeError: Error during sampling, chain 0 returned error code 70, chain 1 returned error code 70, chain 2 returned error code 70, chain 3 returned error code 70